# IP Adapter FaceID

[![Model on HF](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Model-blue)](https://huggingface.co/h94/IP-Adapter-FaceID)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nguu/maze/blob/main/IP_Adapter_FaceID.ipynb)
[![GitHub Repository](https://img.shields.io/github/stars/tencent-ailab/IP-Adapter?style=social)](https://github.com/tencent-ailab/IP-Adapter)

## Enviroment


In [ ]:
import os, shutil

USE_DRIVE = False #@param {type:'boolean'}
if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

%pip install -q diffusers insightface onnxruntime einops accelerate
%pip install -q git+https://github.com/tencent-ailab/IP-Adapter.git
!wget https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin

## Loading IP Adapter Model

In [ ]:
import cv2
import torch
from PIL import Image
from insightface.app import FaceAnalysis
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from ip_adapter.ip_adapter_faceid import IPAdapterFaceID

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

device = 'cuda'
ldm_model_path = 'AVIIAX/majic7.1' #@param {type:'string'}
vae_model_path = 'stabilityai/sd-vae-ft-mse' #@param {type:'string'}
ipa_model_path = 'ip-adapter-faceid_sd15.bin'
device_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
face_analyst = FaceAnalysis(name="buffalo_l", providers=['CPUExecutionProvider'])
face_analyst.prepare(ctx_id=0, det_size=(640, 640))

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)

pipe = StableDiffusionPipeline.from_pretrained(
    ldm_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=AutoencoderKL.from_pretrained(
        vae_model_path,
        torch_dtype=torch.float16,
    ),
    feature_extractor=None,
    safety_checker=None,
)
pipe.to(device)

ip_model = IPAdapterFaceID(pipe, ipa_model_path, device)

## Extract Face Embedding

In [ ]:
SOURCE_IMAGE = '/content/face-demo.jpg' #@param {type:'string'}

src_image = cv2.imread(SOURCE_IMAGE)
src_faces = face_analyst.get(src_image)

faceids = torch.from_numpy(src_faces[0].normed_embedding).unsqueeze(0)

## Generate Images with Face Embedding

In [ ]:
import random

PROMPT = "high quality, masterpiece, landscape, 1girl, white dress" #@param {type:'string'}
NEG_PROMPT = "monochrome, lowres, bad anatomy, worst quality, low quality, blurry, nsfw" #@param {type:'string'}
WIDTH = 512 #@param {type:'integer'}
HEIGHT = 768 #@param {type:'integer'}
BATCH_SIZE = 2 #@param {type:'integer'}
STEPS = 30 #@param {type:'integer'}
SEED = -1 #@param {type:'integer'}

images = ip_model.generate(
    prompt=PROMPT,
    negative_prompt=NEG_PROMPT,
    faceid_embeds=faceids,
    num_samples=BATCH_SIZE,
    width=WIDTH,
    height=HEIGHT,
    num_inference_steps=STEPS,
    seed=random.randint(0, 999999999) if SEED == -1 else SEED,
)

grid = image_grid(images, 1, BATCH_SIZE)
grid

## Save Result Images

In [ ]:
from datetime import datetime

OUTPUT_DIR = '/content/output' #@param {type:'string'}
time = datetime.now().strftime('%Y%m%d-%H%M%S')
for index, image in enumerate(images):
  name = f'{time}-{index}.png'
  image.save(f'{OUTPUT_DIR}/{name}')